## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Amahai,ID,-3.3333,128.9167,83.07,73,41,5.30,scattered clouds
1,1,Gubkinskiy,RU,64.4340,76.5026,58.37,60,83,12.66,broken clouds
2,2,Salisbury,GB,51.0693,-1.7957,60.10,93,77,6.49,broken clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,37.51,75,0,23.02,clear sky
4,4,Fortuna,US,40.5982,-124.1573,55.92,91,90,6.91,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Amahai,ID,-3.3333,128.9167,83.07,73,41,5.30,scattered clouds
22,22,Vaini,TO,-21.2000,-175.2000,77.16,88,75,20.71,moderate rain
23,23,Mahibadhoo,MV,3.7833,72.9667,82.53,72,99,5.06,overcast clouds
25,25,Addanki,IN,15.8167,79.9833,86.05,53,100,13.42,overcast clouds
32,32,Bani,DO,18.2833,-70.3333,78.75,84,16,9.53,few clouds
...,...,...,...,...,...,...,...,...,...,...
700,700,Joshimath,IN,30.5667,79.5667,76.69,79,69,4.88,broken clouds
701,701,Eyl,SO,7.9803,49.8164,77.40,62,100,23.87,overcast clouds
706,706,Pyay,MM,18.8167,95.2167,87.42,67,84,6.85,broken clouds
707,707,Artyom,AZ,40.4723,50.3330,78.94,68,96,8.77,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                263
City                   263
Country                263
Lat                    263
Lng                    263
Max Temp               263
Humidity               263
Cloudiness             263
Wind Speed             263
Current Description    263
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna() # One Nan Value dropped
clean_df.count()

City_ID                263
City                   263
Country                263
Lat                    263
Lng                    263
Max Temp               263
Humidity               263
Cloudiness             263
Wind Speed             263
Current Description    263
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Amahai,ID,83.07,scattered clouds,-3.3333,128.9167,
22,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
23,Mahibadhoo,MV,82.53,overcast clouds,3.7833,72.9667,
25,Addanki,IN,86.05,overcast clouds,15.8167,79.9833,
32,Bani,DO,78.75,few clouds,18.2833,-70.3333,
34,Komsomolskiy,UZ,84.92,scattered clouds,40.4272,71.7189,
36,Cockburn Town,TC,82.33,clear sky,21.4612,-71.1419,
38,Aksu,CN,88.81,overcast clouds,41.1231,80.2644,
47,Salalah,OM,77.09,overcast clouds,17.0151,54.0924,
48,Makakilo City,US,84.52,broken clouds,21.3469,-158.0858,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True) # Replacing blank spaces with NAN values to delete them
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Amahai,ID,83.07,scattered clouds,-3.3333,128.9167,HOTEL ISABELA
22,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
23,Mahibadhoo,MV,82.53,overcast clouds,3.7833,72.9667,Omadhoo Inn
25,Addanki,IN,86.05,overcast clouds,15.8167,79.9833,Shanthi residency
32,Bani,DO,78.75,few clouds,18.2833,-70.3333,Hospedaje Henriquez
...,...,...,...,...,...,...,...
700,Joshimath,IN,76.69,broken clouds,30.5667,79.5667,Shivalik Camps & Resort
701,Eyl,SO,77.40,overcast clouds,7.9803,49.8164,Amiin Hotel
706,Pyay,MM,87.42,broken clouds,18.8167,95.2167,Shwe Pyi Thar
707,Artyom,AZ,78.94,overcast clouds,40.4723,50.3330,Merdan Respect


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Lat/dt><dd>{Lat} °</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))